In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from tqdm import tqdm
import sys
sys.path.append('utils/')

# use non standard flow_from_directory
from image_preprocessing_ver1 import ImageDataGenerator
# it outputs not only x_batch and y_batch but also image names

from keras.models import Model
from xception import Xception, preprocess_input

Using TensorFlow backend.


In [3]:
data_dir = '/home/ubuntu/data/'

In [4]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', 
    target_size=(299, 299),
    batch_size=64, shuffle=False
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', 
    target_size=(299, 299),
    batch_size=64, shuffle=False
)

Found 25600 images belonging to 256 classes.
Found 5120 images belonging to 256 classes.


# Get model and remove the last layer

In [5]:
model = Xception()
model.load_weights('xception_weights.hdf5')
# remove softmax
model.layers.pop()
model = Model(model.input, model.layers[-1].output)
# now model outputs logits

# Save logits as a dict: image name -> logit (256 dimensional vector)

In [6]:
batches = 0
train_logits = {}

for x_batch, _, name_batch in tqdm(train_generator):
    
    batch_logits = model.predict_on_batch(x_batch)
    
    for i, n in enumerate(name_batch):
        train_logits[n] = batch_logits[i]
    
    batches += 1
    if batches >= 400: # 25600/64
        break

399it [04:04,  1.63it/s]

In [7]:
batches = 0
val_logits = {}

for x_batch, _, name_batch in tqdm(val_generator):
    
    batch_logits = model.predict_on_batch(x_batch)
    
    for i, n in enumerate(name_batch):
        val_logits[n] = batch_logits[i]
    
    batches += 1
    if batches >= 80: # 5120/64
        break


0it [00:00, ?it/s]
1it [00:00,  1.63it/s]
2it [00:01,  1.63it/s]
3it [00:01,  1.63it/s]
4it [00:02,  1.63it/s]
5it [00:03,  1.63it/s]
6it [00:03,  1.63it/s]
7it [00:04,  1.63it/s]
79it [00:48,  1.63it/s]

In [8]:
np.save(data_dir + 'train_logits.npy', train_logits)
np.save(data_dir + 'val_logits.npy', val_logits)